In [1]:
import torch
from nca.constsants import H, NUM_CHANNELS, W
from nca.generate_map import generate_training_world
from policy_net.learning_agent import LearningAgent
from policy_net.policy_net import PolicyNet, PolicyNetONNX
from nca.nca_model import NCA

device = 'cuda' if torch.cuda.is_available() else 'cpu' #cuda

model = NCA(NUM_CHANNELS).to(device)


policy0 = PolicyNet(num_species=len(["grass_0", "shrub_0", "tree_0"]), h=H, w=W).to(device)
policy1 = PolicyNetONNX(model_path="policy_net.onnx")
agent0 = LearningAgent(agent_id=0, policy_net=policy0, available_species=["grass_0", "shrub_0", "tree_0"], start_quadrant="top_left")
agent1 = LearningAgent(agent_id=1, policy_net=policy1, available_species=["grass_1", "shrub_1", "tree_1"], start_quadrant="bottom_right")
agents = [agent0, agent1]


In [3]:
import onnx
model = onnx.load("policy_net.onnx")
print(f"Number of initializers: {len(model.graph.initializer)}")
for i, init in enumerate(model.graph.initializer):
    print(f"{i}: {init.name}, shape={init.dims}")


Number of initializers: 8
0: conv1.weight, shape=[32, 16, 3, 3]
1: conv1.bias, shape=[32]
2: conv2.weight, shape=[32, 32, 3, 3]
3: conv2.bias, shape=[32]
4: location_head.weight, shape=[1, 32, 1, 1]
5: location_head.bias, shape=[1]
6: classifier.weight, shape=[3, 32]
7: classifier.bias, shape=[3]


In [2]:
grid = generate_training_world(H, W, seed_plants=False).to(device)
import numpy as np

def log_channel_sums(grid: torch.Tensor):
    assert grid.ndim == 4, "Expected shape [1, channels, height, width]"
    channels = grid.shape[1]
    for c in range(channels):
        channel_sum = grid[0, c].sum().item()
        print(f"Channel {c} sum: {channel_sum}")


log_channel_sums(grid)

Channel 0 sum: 2087.0
Channel 1 sum: 382.0
Channel 2 sum: 827.0
Channel 3 sum: 800.0
Channel 4 sum: 343.4313049316406
Channel 5 sum: 0.0
Channel 6 sum: 0.0
Channel 7 sum: 0.0
Channel 8 sum: 0.0
Channel 9 sum: 0.0
Channel 10 sum: 0.0
Channel 11 sum: 0.0
Channel 12 sum: 0.0
Channel 13 sum: 0.0
Channel 14 sum: 0.0


In [3]:
grid.shape

torch.Size([1, 15, 64, 64])

In [2]:
from nca.constsants import CHANNELS
from policy_net.animate import animate_species_ownership_with_static_layers
from policy_net.environment import Environment
from policy_net.game import play_game


print(f"episode {1}")
grid = generate_training_world(H, W, seed_plants=False).to(device)
ownership_grid = torch.full((1, H, W), fill_value=-1, dtype=torch.long, device=device)
elevation_static = grid[:, CHANNELS["elevation"]].clone().detach()
soil_static = {
    idx: grid[:, idx].clone().detach()
    for idx in CHANNELS["soil"].values()
}
env = Environment(grid, model, ownership_grid, agents, elevation_static, soil_static, steps_per_turn=5)

scores = play_game(env, max_turns=15,save=True)
print(scores)
ani = animate_species_ownership_with_static_layers(env, elevation_static, soil_static)
from IPython.display import HTML
HTML(ani.to_jshtml())


episode 1
savinggg
torch.Size([1, 16, 64, 64])
(0, 'shrub_0', 34, 22)
savinggg
torch.Size([1, 16, 64, 64])
(1, 'shrub_1', 46, 62)
(0, 'tree_0', 22, 2)
(1, 'shrub_1', 14, 38)
(0, 'tree_0', 22, 58)
(1, 'shrub_1', 50, 14)
(0, 'grass_0', 10, 2)
(1, 'grass_1', 42, 26)
(0, 'tree_0', 62, 10)
(1, 'tree_1', 34, 22)
(0, 'tree_0', 18, 14)
(1, 'grass_1', 42, 42)
(0, 'tree_0', 62, 2)
(1, 'shrub_1', 46, 26)
(0, 'tree_0', 62, 18)
(1, 'tree_1', 6, 62)
(0, 'grass_0', 42, 50)
(1, 'grass_1', 42, 14)
(0, 'shrub_0', 50, 62)
(1, 'tree_1', 30, 38)
(0, 'grass_0', 18, 22)
(1, 'tree_1', 6, 2)
(0, 'tree_0', 26, 10)
(1, 'shrub_1', 14, 42)
(0, 'tree_0', 2, 46)
(1, 'shrub_1', 30, 42)
(0, 'tree_0', 26, 42)
(1, 'shrub_1', 26, 10)
(0, 'grass_0', 34, 2)
(1, 'shrub_1', 2, 58)
{0: 402.0, 1: 33.0}
